# Dataset Setup

IMPORTANT:

This will create more than 500 images.

It needs to be run ONCE ONLY!!!!

Unless you need a new dataset, but make sure to update the path.

### Imports

In [3]:
import sys
import os
from config import Config
sys.path.append(Config.root_path)

from Python.data_processing.image_tools import create_greyscale_masks
from Python.data_processing.image_tools import split_png_images_in_four_from_dir
from Python.data_processing.image_tools import load_image_dir_to_array
from Python.data_processing.utils import create_weight_map
from Python.data_processing.save_model import write_dict_to_disk

### Step 1
Covert masks from categorical RGB encoding to categorical greyscale encoding 

In [4]:
# Path to directory containing RGB encoded segmentation masks
rgb_mask_path = os.path.join(
    Config.root_path,
    r"Data/Priddy_raw/SegmentationClass"
)
# Path to output directory for categorical greyscale segmentation masks
output_mask_path = os.path.join(
    Config.root_path,
    r"Data/Priddy_processed/SegmentationClass_categorical"
)
# Path to json file containing class <-> RGB conversions
class_rgb_path = os.path.join(
    Config.root_path,
    r"Data/Priddy_raw/class_RGB.json"
)
# Path to json file containing class <-> greyscale conversions
class_greyscale_path = os.path.join(
    Config.root_path,
    r"Data/Priddy_raw/class_categorical.json"
)
create_greyscale_masks(
    rgb_mask_path, output_mask_path, class_rgb_path, class_greyscale_path
)

### Step 2
Split images into 4

In [6]:
# Path to directory containing full size images
image_dir_path = os.path.join(
    Config.root_path, 
    r"Data/Priddy_raw/JPEGImages"
)
# Path to output directory for split images
output_dir_path = os.path.join(
    Config.root_path, 
    r"Data/Priddy_raw/JPEGImages_512"
)
split_png_images_in_four_from_dir(image_dir_path, output_dir_path)

### Step 3
Create class weights from raw images and save then to a json file

In [7]:
# Path to directory containing all greyscale categorical segmentation 
# masks
segmentation_dir_path = os.path.join(
    Config.root_path, 
    r"Data/Priddy_processed/SegmentationClass_categorical"
)
# Path to output location for weight map json file
weight_map_path = os.path.join(
    Config.root_path,
    r"Data/Priddy_processed/weights.json"
)
mask_array, mask_filenames = load_image_dir_to_array(
    segmentation_dir_path,
    sorted=True
)
weight_map = create_weight_map(mask_array)
write_dict_to_disk(weight_map, weight_map_path)